In [1]:
import scanpy as sc
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.pyplot import rc_context
import matplotlib as mpl
import matplotlib.pyplot as plt




In [2]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi = 160, dpi_save = 180, 
                              vector_friendly = True, format = 'svg', facecolor="white")

scanpy==1.8.2 anndata==0.7.6 umap==0.5.2 numpy==1.21.2 scipy==1.7.1 pandas==1.3.3 scikit-learn==1.0.1 statsmodels==0.13.0 python-igraph==0.9.6 pynndescent==0.5.4


In [3]:
rna = sc.read("/Users/GoksuAvar/Desktop/TBproject/Lung_TB_T_Cells_CITESEQ_RNA.h5ad")
protein = sc.read("/Users/GoksuAvar/Desktop/TBproject/Lung_TB_T_Cells_CITESEQ_Protein.h5ad")
adata = rna
adata.obsm["protein_expression"] = protein
adata.var_names_make_unique()
adata.raw = adata
adata

AnnData object with n_obs × n_vars = 500089 × 33538
    obs: 'cell_id', 'nUMI', 'nGene', 'percent_mito', 'batch', 'TB_status', 'UMAP_1', 'UMAP_2', 'cluster_name', 'cluster_ids', 'donor'
    obsm: 'protein_expression'

In [4]:
adata.layers["counts"] = adata.X.copy()

In [5]:
adata.obsm["protein_expression"] = adata.obsm["protein_expression"].X.toarray()

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20)

normalizing counts per cell


In [ ]:
sc.pl.highest_expr_genes(protein, n_top=31)

In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('MT-')
adata.var['ribo'] = adata.var_names.str.startswith('RPS','RPL')
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt', "ribo"], percent_top=None, log1p=False, inplace=True)



In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt', "pct_counts_ribo"],
             jitter=0.1, multi_panel=True, size=1)

In [ ]:
sc.pl.scatter(adata, x="UMAP_1", y="UMAP_2", color = "pct_counts_mt", color_map="RdPu")
sc.pl.scatter(adata, x="UMAP_1", y="UMAP_2", color = "percent_mito", color_map="RdPu")
sc.pl.scatter(adata, x="UMAP_1", y="UMAP_2", color = "pct_counts_ribo", color_map="RdPu")

In [ ]:
sc.pl.scatter(adata, x="UMAP_1", y="UMAP_2", color = "total_counts", color_map="RdPu")
sc.pl.scatter(adata, x="UMAP_1", y="UMAP_2", color = "n_genes_by_counts", color_map="RdPu")

In [ ]:
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata, x='total_counts', y='pct_counts_ribo')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts', color="pct_counts_mt", color_map="RdPu")

In [ ]:
sns.histplot(adata.obs["n_genes_by_counts"], kde=True)


In [ ]:
sns.histplot(adata.obs["total_counts"], kde=True)

In [ ]:
sns.histplot(adata.obs["pct_counts_mt"], kde=True)

In [ ]:
sns.histplot(adata.obs["pct_counts_ribo"], kde=True)

In [ ]:
protein.var["control"] = protein.var_names.str.contains("Mouse")
sc.pp.calculate_qc_metrics(protein,percent_top=(5, 10, 15), var_type="antibodies", 
                           qc_vars=("control",),inplace=True)

In [ ]:
sns.jointplot("log1p_total_counts", "n_antibodies_by_counts", protein.obs, kind="hex", norm=mpl.colors.LogNorm())
sns.jointplot("log1p_total_counts", "log1p_total_counts_control", protein.obs, kind="hex", norm=mpl.colors.LogNorm())

In [ ]:
scrublet_predicted = pd.read_csv("/Users/GoksuAvar/Desktop/TBproject/Scrublet_predicted.tsv", sep="\t")
scrublet_predicted.set_axis(adata.obs_names, axis=0)
scrublet_predicted = scrublet_predicted.assign(scrublet_predicted_doublets_binary = [0 if 
                                                                                     i == False 
                                                                                     else 1 
                                                                                     for i in scrublet_predicted['scrublet_predicted_doublets']])
scrublet_predicted

In [ ]:
adata.obs["scrublet_doublet_scores"] = scrublet_predicted.scrublet_doublet_scores.set_axis(adata.obs_names, axis=0)
adata.obs["scrublet_predicted_doublets"] = scrublet_predicted.scrublet_predicted_doublets.set_axis(adata.obs_names, axis=0)
adata.obs["scrublet_predicted_doublets_binary"] = scrublet_predicted.scrublet_predicted_doublets_binary.set_axis(adata.obs_names, axis=0)

In [ ]:
scrublet_predicted.scrublet_doublet_scores.shape

In [ ]:
adata.obs.scrublet_predicted_doublets_binary.describe()

In [ ]:
sc.pl.scatter(adata, x="UMAP_1", y="UMAP_2", color="scrublet_doublet_scores", color_map="RdPu")
sc.pl.scatter(adata, x="UMAP_1", y="UMAP_2", color="scrublet_predicted_doublets_binary", color_map="RdPu")


In [ ]:
adata_scr = adata[adata.obs.scrublet_predicted_doublets_binary == 0, :]

In [ ]:
adata_scr

In [ ]:
sc.pl.violin(adata_scr, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt', "pct_counts_ribo"],
             jitter=0.1, multi_panel=True, size=1)

In [ ]:
solo_predicted = pd.read_csv("/Users/GoksuAvar/Desktop/TBproject/SOLO_predicted.tsv", sep="\t")
solo_predicted.set_axis(adata.obs_names, axis=0)
solo_predicted

In [ ]:
adata.obs["solo_doublet_scores"] = solo_predicted.doublet.set_axis(adata.obs_names, axis=0)
adata.obs["solo_singlet_scores"] = solo_predicted.singlet.set_axis(adata.obs_names, axis=0)

In [ ]:
adata.obs.solo_doublet_scores.describe()

In [ ]:
adata.obs.solo_singlet_scores.describe()

In [ ]:
sc.pl.scatter(adata, x="UMAP_1", y="UMAP_2", color="solo_doublet_scores", color_map="RdPu")
sc.pl.scatter(adata, x="UMAP_1", y="UMAP_2", color="solo_singlet_scores", color_map="RdPu")



In [ ]:
#gene_list = ['IL2RA', 'CD8A', 'CD4', "KLRB1", "DPP4", "PTGDR2", "IL22", "IL17A"]

#for i in gene_list:
 #   sc.pl.scatter(adata, x="UMAP_1", y="UMAP_2", color=i, color_map="RdPu")

In [ ]:
adata.obs["batch"] = adata.obs["batch"].astype("category") 

In [ ]:
protein.obs["batch"] = protein.obs["batch"].astype("category") 

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)

In [ ]:
with rc_context({'figure.figsize': (12, 4)}):
    sc.pl.violin(protein, keys=["CD4_protein"], groupby="batch")

In [ ]:
with rc_context({'figure.figsize': (12, 4)}):
    sc.pl.violin(adata, keys=["n_genes_by_counts"], groupby="batch", use_raw=True, log=False)

In [ ]:
with rc_context({'figure.figsize': (4, 4)}):
    sc.pl.violin(adata, keys=["n_genes_by_counts", "total_counts", 'pct_counts_mt', "pct_counts_ribo"], groupby="TB_status", use_raw=True, log=False, multipanel=True)

In [ ]:
adata.obs["donor"].describe()

In [ ]:
print(adata.obs.solo_doublet_scores.describe())
fig = sns.histplot(adata.obs["solo_doublet_scores"], kde=True)
mean = adata.obs["solo_doublet_scores"].mean()
std = adata.obs["solo_doublet_scores"].std()
fig.axvline(mean, color='r', linestyle='--')
fig.axvline(mean+2*std, color='g', linestyle='--')


In [ ]:
sc.pl.scatter(adata[adata.obs["solo_doublet_scores"]>(mean+2*std),:], x="UMAP_1", y="UMAP_2", 
              color="solo_doublet_scores", color_map="viridis", size=120000/500089)


In [ ]:
print(adata.obs.n_genes_by_counts.describe())
fig = sns.histplot(adata.obs["n_genes_by_counts"], kde=True)
mean = adata.obs["n_genes_by_counts"].mean()
std = adata.obs["n_genes_by_counts"].std()
fig.axvline(mean, color='r', linestyle='--')
fig.axvline(mean+2*std, color='g', linestyle='--')


In [ ]:
sc.pl.scatter(adata[adata.obs["n_genes_by_counts"]>(mean+2*std),:], x="UMAP_1", y="UMAP_2", size=120000/500089)


In [ ]:
sc.pl.scatter(adata, x="UMAP_1", y="UMAP_2", color="batch", color_map="RdPu")


In [ ]:
for i in len(range(adata.obs["batch"])):
    with rc_context({'figure.figsize': (2, 2)}):
        sc.pl.scatter(adata[adata.obs["batch"]==i,:], x="UMAP_1", y="UMAP_2", size=120000/500089)
